# R-Style Custom Infix Operators in Python

This brief notebook shows how to implement custom infix operators in Python.
The method I use to do this is basically a hack and probably shouldn't be used in any serious code.
Nonetheless, it does serve as an interesting illustration of some cool core Python features: dunder methods and decorators.

As the title says, we will be implementing 'R-style' infix operators.
What does this mean?
Well, in R you can define custom infix operators merely by using a special naming convention.
I'll show you how to do that shortly.
But first I just want to mention that by using `rpy2`, we can freely intermingle Python and R code in one notebook.
We just have to use the `%%R` cell magic to denote an R cell, as I do below.
(In this notebook I don't actually import `rpy2`.
Rather, I load a script I found called `RWinOut` that fixes a Windows bug with the `rpy2` output.)

Here's how we can make a very basic version of magrittr's pipe operator.
In case you're not familiar, the pipe operator allows you to 'pipe' values left to right through functions.

In [1]:
# set up R
%load_ext RWinOut

In [2]:
%%R
`%>%` <- function(x, f) {
    f(x)
}

square <- function(x) {
    x ^ 2
}

2 %>% square %>% square

[1] 16


The rule for defining an infix operator is that the function name must begin and end with a percent sign,
and the name itself must be surrounded in backticks.
Then R allows you to freely use the function as an infix operator.

There's no obvious or nice way to do this in Python, but we can hack our way to something similar.
The trick is that we can override the meaning of the `%` symbol using the `__mod__` and `__rmod__` dunder methods.
In case you're not familiar, a dunder method is any method whose name begins and ends with <b>d</b>ouble <b>under</b>scores.
Dunder methods allow us to implement custom behavior for basically every bit of 'special' syntax in Python.

Every operator (`+`, `*`, `-`, `/`, `%`, `<`, `>`, etc.) has behavior defined by two dunder methods,
for example `__mod__` and `__rmod__` for `%`.
If `x` is an instance of a class with a `__mod__` method, then `x % y` calls `x.__mod__(y)`.
Similarly, if `y` does not have its own `__mod__` method that works with objects like `x`, then `y % x` calls `x.__rmod__(y)`.

This is all we need to implement infix operators!
Check it out.

In [3]:
class Infix:
    def __init__(self, func):
        self.func = func
    
    def __mod__(self, other):
        return self.func(self.left, other)
    
    def __rmod__(self, other):
        self.left = other
        return self
    
def add(x, y):
    return x + y
add = Infix(add)

2 %add% 2

4

So how does this work exactly?

We create a class `Infix` that implements the `__mod__` and `__rmod__` methods.
To create a new `Infix` object, we pass in a function of two arguments which then gets stored in the `func` attribute.
When we write an expression like `2 %add% 2`, Python reads left to right and first evaluates `2 % add`.
Although 2 is an integer and thus has `%` defined for it, it doesn't work for objects of type `Infix`,
so `add`'s `__rmod__` method gets called.
This method just saves 2 into `add.left`, named so because it is the left argument, and returns `add`.
Since `2 % add` evaluates to `add`, then `add % 2` gets evaluated.
This calls `add.__mod__` which then calls the stored function using the previously stored left argument
and the newly received right argument (called `other`).

Pretty neat, huh?

To make a function into an infix operator, we just define the function and then call `Infix` on it.
This is exactly the use case for another Python feature called 'decorators.'
Basically:

```
@decorate
def f(...):
    ...
```

is equivalent to

```
def f(...):
    ...
f = decorate(f)
```

So we can do the following:

In [4]:
@Infix
def p(x, f): # p for pipe
    return f(x)

def square(x):
    return x ** 2

2 %p% square %p% square

16

This is basically as nice as the R version.
There are some clear limitations, however.
Python doesn't allow you to give your function a name like `>`,
so we can never exactly recreate the `%>%` function using this method.
Also, since `2 %p% square` is really `(2 % p) % square`, we're stuck with the precedence of `%`, which is the same as `*` and `/`.
However, there's nothing special about `%`. I was just copying R's syntax.
If you wanted different syntax or different precedence, you could use a different operator.

Anyway, as a final trick, let's implement a basic infix function composition.

In [5]:
@Infix
def o(f, g): # looks like the composition symbol
    return lambda x: f(g(x))

# compose functions on the fly in a call to map
str_list = [' HELLO  ', '  TheRe! ', 'how ', 'are', '      YOU?']
print(' '.join(map(str.lower %o% str.strip, str_list)))

# the above is not really necessary because of list comprehensions / generator expressions
# but composition is still really cool
print(' '.join(s.strip().lower() for s in str_list))

hello there! how are you?
hello there! how are you?
